In [1]:
import os
import sys
import numpy as np
from datetime import date


#Better to define most functions outside and call them(easier)
sys.path.append('../modules/')
import functions as f

f.prepare_pipeline()

## THINGS THAT STILL NEED TO BE DONE:

Refining all the options of the DM ( building a more general while loop to control the directions, define specific functions to make it more simple, contemplate possible situations and make the slot filling more flexible)

Use neuralcoref or SMTH to be able to extract the subject (pronoun or name at least) so the answers can be more personalized.

Recognize more tipes of intents?? Expand the matcher

Add slots (for example the travelling options??)

Do tests to see what slot matching or intent matching fails and add it to the structures

Change response generation!!!!! More random etc.. what options do we have?

Link to the database and define profiles better

In [2]:
#This piece of code was just to try it out -  maybe it helps as an example but it's not part of the DM

text= u"Hello Cora. My uncle William is 65 years old and he smokes. Can you give me the risk level? He had a heart transplant two years ago and he has diabetis"
#Define intents dictionary
Intents={'greeting': False, 'thank': False, 
          'accept': False,'deny':False,'goodbye': False, 'ask_help':False }
    
    
#Define  frame dictionary
Frame={'age': 0,'smoker':False,'med_cond_risk':[],
       'med_cond':[],'live_in': 0,'date': date.today().strftime("%d/%m/%Y"), 'pronoun':0,'name':0}

Frame, Intents = f.intent_slot_filling(text, Frame, Intents)

Hello Cora. My uncle William is 65 years old and he smokes. Can you give me the risk level? He had a heart transplant two years ago and he has diabetis
{'age': '65', 'smoker': True, 'med_cond_risk': [2, 1], 'med_cond': [['heart', 'transplant'], ['has', 'diabetis']], 'live_in': 0, 'date': '09/12/2020', 'pronoun': 0, 'name': 'William'}
{'greeting': True, 'thank': False, 'accept': False, 'deny': False, 'goodbye': False, 'ask_help': False}


In [3]:
#start code for Cora DM


#Define the stages of the dialogue manager with a vector and 0,1 so we have memory on which stage are we and what things
#have been done before (I thought it could be useful)

DM_vec=[0,0,0]

#Define intents dictionary: this dictionary has to be initialized everytime ( we use a function to initialize it)
Intents = f.init_intent()
    
#Define  frame dictionary: this dictionary accumulates the info during the whole conversation

Frame= f.init_frame()

#IDEA: create a function that notices during the whole conversation if any particular or important intent is happening??
#      not only look for intent when we expect one.


# Greeting part
while DM_vec[0] == 0:
    
    print('Cora presentation and greeting')
    input_text = f.wait_input()
    Frame, Intents = f.intent_slot_filling(input_text,Frame,Intents)
    print(Intents)
    
    if Intents['ask_help'] == True:
        print('Of course I will help!')
    
    if Intents['greeting'] == True:
        print('Nice to meet you')
        #Idea: if neuralcoref works it could say the name if you gave your name!
    
    DM_vec[0] = 1
    

#Second stage: where the required slots are filled    
while DM_vec[1]== 0:
    
    have_age= False
    have_loc = False
    
    if (Frame['age'] != 0) and (f.is_number(Frame['age']) == True):
        have_age = True
        
    #Here we need to check if the country is in our list
    if (Frame['live_in'] != 0):
        have_loc = True
    
    #Here the subject is missing but it needs to be added
    if (have_age == True) and (have_loc == True):
        DM_vec[1] = 1
        break
    else:
        print(" I will need to ask you a few more questions to fill the query up")
    
        if (Frame['age'] == 0) or (f.is_number(Frame['age']) == False):
            print("Sorry I didn't quite get your/her/his age")
            input_text = f.wait_input()
            Frame, Intents = f.intent_slot_filling(input_text,Frame,Intents)
            Intents = f.init_intent()
        else:
            have_age =True


        #Here we need to check if the country is in our list
        if (Frame['live_in'] == 0):
            print(" Sorry the location you specify is either not a country or is not in our list")
            input_text = f.wait_input()
            Frame, Intents = f.intent_slot_filling(input_text,Frame,Intents)
            Intents = f.init_intent()
        else:
            have_loc = True
            
            
#############################################
# Part where you extract the info from the database and fill in the profile

for k,v in Frame.items():
    if  (type(v)!= list) and ((v != 0) or (v == True)):
        print('slots that are filled:', k, 'values:', v)
    elif (type(v)== list) and (len(v) != 0):
        print('slots that are filled:', k, 'values:', v)
        
#Check database

print("Your profile is...")
#############################################


#for now it tries to fill all the slots, maybe it would be better if it's more flexible
while DM_vec[2]==0:
    
    
    ######## This should be turned into a function??
    empty_slots=[]
    for k,v in Frame.items():
        if  (type(v)!= list) and (type(v)!= bool) and (v == 0) and (k != 'pronoun') and (k != 'name'):
            empty_slots.append(k)
            print('slots that are NOT filled:', k)
            if k == 'smoker':
                print(v)
        elif (type(v)== list) and (len(v) == 0) and (k != 'med_cond'):
            empty_slots.append(k)
            print('slots that are NOT filled:', k)
    
            
    if len(empty_slots) == 0:
        DM_vec[2]=1
        
    else: 
        for item in empty_slots:
            print("I need information about {}".format(item))
            input_text = f.wait_input()
            Frame, Intents = f.intent_slot_filling(input_text,Frame,Intents)
            Intents = f.init_intent()
            

#!!!!!If we did manage to get more information:
# here we would repeat the connection to the database and the profiling, otherwise we continue

#This part tries to see if we need another profiling or we say goodbye
while DM_vec[3] == 0:
    print('Would you like to know the profile of another person? Maybe your sister, your uncle..')
    
    input_text = f.wait_input()
    Frame, Intents = f.intent_slot_filling(input_text,Frame,Intents)
    
    if Intents['accept'] == True:
        
        print('Lets start another query then')
        DM_vec[3] = 1
        #HERE WE NEED TO FIND A WAY TO GO BACK TO THE START: we need a higher level while loop to control this flow
    
    if (Intents['deny'] == True) or (Intents['goodbye']== True):
        
        print('As you want. I hope I see you soon. Stay safe!')
        DM_vec[3] = 1
    
    
        
    
        
    
    



Cora presentation and greeting


Enter your sentence:  hi cora


hi cora
{'age': 0, 'smoker': 0, 'med_cond_risk': [], 'med_cond': [], 'live_in': 0, 'date': '09/12/2020', 'pronoun': 0, 'name': 0}
{'greeting': True, 'thank': False, 'accept': False, 'deny': False, 'goodbye': False, 'ask_help': False}
{'greeting': True, 'thank': False, 'accept': False, 'deny': False, 'goodbye': False, 'ask_help': False}
Nice to meet you
 I will need to ask you a few more questions to fill the query up
Sorry I didn't quite get your/her/his age


Enter your sentence:  I am 25


I am 25
{'age': '25', 'smoker': 0, 'med_cond_risk': [], 'med_cond': [], 'live_in': 0, 'date': '09/12/2020', 'pronoun': 0, 'name': 0}
{'greeting': False, 'thank': False, 'accept': False, 'deny': False, 'goodbye': False, 'ask_help': False}
 Sorry the location you specify is either not a country or is not in our list


...

i live in barcelona
{'age': '25', 'smoker': 0, 'med_cond_risk': [], 'med_cond': [], 'live_in': 'barcelona', 'date': '09/12/2020', 'pronoun': 0, 'name': 0}
{'greeting': False, 'thank': False, 'accept': False, 'deny': False, 'goodbye': False, 'ask_help': False}
slots that are filled: age values: 25
slots that are filled: live_in values: barcelona
slots that are filled: date values: 09/12/2020
Your profile is...
slots that are NOT filled: smoker
0
slots that are NOT filled: med_cond_risk
I need information about smoker


Enter your sentence:  I am not a smoker


I am not a smoker
{'age': '25', 'smoker': False, 'med_cond_risk': [], 'med_cond': [], 'live_in': 'barcelona', 'date': '09/12/2020', 'pronoun': 0, 'name': 0}
{'greeting': False, 'thank': False, 'accept': False, 'deny': False, 'goodbye': False, 'ask_help': False}
I need information about med_cond_risk


Enter your sentence:  I have diabetis


I have diabetis
{'age': '25', 'smoker': False, 'med_cond_risk': [1], 'med_cond': [['have', 'diabetis']], 'live_in': 'barcelona', 'date': '09/12/2020', 'pronoun': 0, 'name': 0}
{'greeting': False, 'thank': False, 'accept': False, 'deny': False, 'goodbye': False, 'ask_help': False}
